In [2]:
import pandas as pd
import numpy as np

In [6]:
data_df = pd.read_csv("data/2017-09/2017-09-13istdaten.csv", sep=";")

/home/student/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [187]:
data_df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
0,13.09.2017,80:06____:17010:000,80:06____,DB,DB Regio AG,Zug,17010,RE,NaN,RE,...,False,8500090,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 05:45,13.09.2017 05:45:00,PROGNOSE,False
1,13.09.2017,80:06____:17012:000,80:06____,DB,DB Regio AG,Zug,17012,RE,NaN,RE,...,False,8500090,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 06:34,13.09.2017 06:34:00,PROGNOSE,False
2,13.09.2017,80:06____:17013:000,80:06____,DB,DB Regio AG,Zug,17013,RE,NaN,RE,...,False,8500090,Basel Bad Bf,13.09.2017 06:25,13.09.2017 06:28:00,PROGNOSE,NaN,NaN,PROGNOSE,False
3,13.09.2017,80:06____:17014:000,80:06____,DB,DB Regio AG,Zug,17014,RE,NaN,RE,...,False,8500090,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 09:48,13.09.2017 09:48:00,PROGNOSE,False
4,13.09.2017,80:06____:17015:000,80:06____,DB,DB Regio AG,Zug,17015,RE,NaN,RE,...,False,8500090,Basel Bad Bf,13.09.2017 08:06,13.09.2017 08:11:00,PROGNOSE,NaN,NaN,PROGNOSE,False


In [90]:
# Extract list of stop from metadata file.
with open("data/BFKOORD_GEO") as file:
    metadata = file.readlines()
    
metadata_cleaned = [line.split("%") for line in metadata]
metadata_cleaned = [[line[0].split(), line[1][1:-1]] for line in  metadata_cleaned]
metadata_cleaned = [[line[0][0], line[0][1], line[0][2], line[0][3], line[1]] for line in metadata_cleaned]

metadata_df = pd.DataFrame(metadata_cleaned, columns=["StationID", "Longitude", "Latitude", "Height", "Remark"])

metadata_df["Longitude"] = pd.to_numeric(metadata_df["Longitude"])
metadata_df["Latitude"] = pd.to_numeric(metadata_df["Latitude"])

In [173]:
def distance_from_gps_coord(s_lat, s_lon, e_lat, e_lon):
    """
        Compute distance from two points in gps coordinates.
        Output distance in kilometers.
    """
    s_lat = np.radians(s_lat)
    s_lon = np.radians(s_lon)
    e_lat = np.radians(e_lat)
    e_lon = np.radians(e_lon)
    
    return 6371.01 * np.arccos(np.sin(s_lat)* np.sin(e_lat) + np.cos(s_lat)* np.cos(e_lat)* np.cos(s_lon - e_lon))

In [186]:
# Retrieve stops near Zurich main station
zurichHB = metadata_df[metadata_df.Remark == "Zürich HB"]

metadata_df["dist"] = metadata_df.apply(lambda x: distance_from_gps_coord(zurichHB.Latitude, zurichHB.Longitude, x.Latitude, x.Longitude), axis=1)
zurich_stops_df = metadata_df[metadata_df["dist"] < 10]
zurich_stops.head()